In [6]:
import pandas as pd
import random
import scipy as sp
import numpy as np
import cvxpy as cvx
import mosek
import matplotlib.pyplot as plt
import math
import warnings
import parameters
import simulations

#### Load Data

In [7]:
dfTripData = pd.read_csv("Data/location_data.csv")
dfLoadData = pd.read_csv("Data/use_data_2wk.csv") #365 days, 1 min increment, 142 houses 
dfGenData = pd.read_csv("Data/annual_capfac_15MinInterpolation.csv") #representative solar day, 15 min interval
#DAM
dfAsRDdamPrices=pd.read_csv("Data/AS_Prices/DAM/20190701_20190801_PRC_AS_DAM_REGDOWN.csv")
dfAsRUdamPrices=pd.read_csv("Data/AS_Prices/DAM/20190701_20190801_PRC_AS_DAM_REGUP.csv")

#RDU/RU RTM
dfAsRDrtmPrices=pd.read_csv("Data/AS_Prices/RTM/20190701_20190731_PRC_INTVL_AS_RTM_REGDOWN.csv")
dfAsRUrtmPrices=pd.read_csv("Data/AS_Prices/RTM/20190701_20190731_PRC_INTVL_AS_RTM_REGUP.csv")  
dfCostElectric = pd.read_csv("Data/TOU_Rate.csv") #15 min interal 4-9 pm, I think this is tou B

#### Script Parameters

In [8]:
intNumHouseNodes = 5
intNumCommNodes = 2
intTotalNodes = intNumHouseNodes + intNumCommNodes
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh
fltChargeRate = 1.9 # kW
fltDt = 0.25 # delta T
fltHomeRate = 7 # kW
fltWorkRate = 7 # kW

# time and conversion intervals
int_minutes=int(15)
int_run_days=2 # day
int_run_hours=24 # hours
int_run_time_interval=60/int_minutes #
time_kwh=15/60  # convert kw data to kwh


# cost to charge
lsCostElectric = []
lsHolder = dfCostElectric.values.tolist()
for ind in range(int_run_days):
    for n in lsHolder:
        lsCostElectric.append(n[0])

In [9]:
# call functions to get Nodes Dataframe
dfNodesTrips, intVeHouses = parameters.createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData)

#create df of loads for commercial & residential nodes
arrHouseLoads,dfNodeLoads, PeakTimes,PeakLoads, \
lsNodes,dctHomeNode = parameters.convertLoadData(dfLoadData,dfNodesTrips, \
                                    int_run_days,int_run_hours,int_minutes, intVeHouses,intNumCommNodes,intNumHouseNodes)

#create generation for 15 min increments for each node
arrHouseGen,dfNodeGens,npGenData = parameters.SolarGenData(dfGenData,int_run_days,int_run_hours,int_minutes, \
                                              intVeHouses,intNumCommNodes,intNumHouseNodes, dctHomeNode)


# now call function to get the SOC constraints array
arrSOCconstraint = parameters.getSOCconstraints(dfNodesTrips,\
                dfTripData, fltCarEff, fltChargeRate, fltDt, int_run_days)

# # call function to get an array of maximum charging rates and map to location by node
arrChargeRate, arrCanCharge,\
arrChargeLoc, arrHomeStation = parameters.MapCarNodesTime(int_minutes,\
                               int_run_hours,int_run_days, intVeHouses,dfNodesTrips,\
                               dfTripData, fltHomeRate, fltWorkRate)


# # call function to get initial SOCs of vehicles
# # replaces first column of soc constraint array
arrSOCconstraint = parameters.getInitialSoc(arrSOCconstraint, fltBatteryCap)  #should this have a different name?

# # call function to get power consumption of the vehicle
arrConsumptionFinal = parameters.findVehicleConsumption(dfNodesTrips, dfTripData, fltCarEff, \
                fltDt, int_run_days)

# #Get Ancillarys service values, the dict has 31 days for dam ru/rd and rtm ru/rd! can do a full run if we want
dctASallprices,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd = parameters.processAsValues(dfAsRDdamPrices,dfAsRUdamPrices,dfAsRDrtmPrices,dfAsRUrtmPrices,int_run_days)

dfNetNodeLoad,arrNetHomeLoad,\
dctNodeIdentity, dctResIdentity = parameters.netLoadChargeLoc(arrHouseGen,arrHouseLoads,\
                                arrChargeLoc,dfNodeLoads,dfNodeGens,intVeHouses,\
                                intTotalNodes,lsNodes,int_minutes,int_run_hours, \
                                int_run_days, arrHomeStation)

#defaults to .1 dispatch, .5 means half the time its Ru and half the time its Rd, over .5 it increases %Ru
lsRuIdentity, lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval, 0.1)

#save whatever this parameter runs trip ID info is
dfNodesTrips.to_csv('Output/Tripidentity.csv')


#### Vehicle AS Dispatch

In [10]:
constraints, varRegDown,varRegUp,\
varCharge,varNumberOfCycles, varDegradationCost= simulations.make_dispatch_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,\
                        dctResIdentity,lsNodes, arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,lsDAMrdMax,lsDAMruMax,\
                     lsRuIdentity, lsRdIdentity)
obj_value, varRegUp, varRegDown, varCharge =  simulations.make_dispatch_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd, fltDt)
    
# convex usually needs to be minimized
dispatchObj = cvx.Maximize(obj_value)
dispatchProb = cvx.Problem(dispatchObj, constraints)

dispatchProb.solve(solver=cvx.MOSEK)

599.5683063125181

In [11]:
#values to save

fltObjective = obj_value.value
arrRuDispatches = varRegUp.value*lsRuIdentity  
arrRdDispatches = varRegDown.value*lsRuIdentity
arrDegradation = varDegradationCost.value # this is for each vehicle. sum? 
arrNumberCycles = varNumberOfCycles.value
fltRegUpVal = np.sum(varRegUp.value*lsDAMruMax)
fltRegDownVal = np.sum(varRegDown.value*lsDAMrdMax)

In [12]:
#loop for dispatch quantity

lsObj=[]
lsCycles=[]
lsDegradation=[]
lsRegUpVal=[]
lsRegUpDispatchVal=[]
lsRegDownVal=[]
lsRegDownDispatchVal=[]

dfNumberofCycles=pd.DataFrame()
dfDegradation=pd.DataFrame()


for j in np.linspace(0,1,11):
    j=round(j,1)
    print(j)
    lsRuIdentity,lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval,j)
    constraints,varRegDown,varRegUp,\
    varCharge,varNumberOfCycles, varDegradationCost= simulations.make_dispatch_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,\
                        dctNodeIdentity,lsNodes, arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,lsDAMrdMax,lsDAMruMax,\
                     lsRuIdentity, lsRdIdentity)
    obj_value, varRegUp, varRegDown, varCharge =  simulations.make_dispatch_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd, fltDt)
    
    dispatchObj = cvx.Maximize(obj_value)
    dispatchProb = cvx.Problem(dispatchObj, constraints)
    dispatchProb.solve(solver=cvx.MOSEK)
    
    
    ##File outputs
    
    #full arrays, export as csv directly
    arrRegUp=np.array(varRegUp.value)
    arrRegDown=np.array(varRegDown.value)
    arrCharging=np.array(varCharge.value)
    arrRegDownDispatch=varRegDown.value*lsRdIdentity
    arrRegUpDispatch=varRegUp.value*lsRuIdentity
    
        #output array files
    np.savetxt(('Output/Dispatch/Charging'+str(j)+'.csv'),arrCharging, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegUp'+str(j)+'.csv'),arrRegUp, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegDown'+str(j)+'.csv'),arrRegDown, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    np.savetxt(('Output/Dispatch/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    
    #lists by each vehicle, add as an insert to datatframe
    lsCycles=varNumberOfCycles.value
    lsDegradation=varDegradationCost.value 
    
    #dataframe of list values
    dfNumberofCycles.insert(int(j*10),"Cycles"+str(j),lsCycles,True)
    dfDegradation.insert(int(j*10),"Degradation"+str(j),lsDegradation,True)
   
    #Individual values to save as list, these will zip into dataframe outside of loop
    lsObj.append(obj_value.value)
    lsRegUpVal.append(np.sum(np.sum(arrRegUp,axis=0)*lsDAMruMax))
    lsRegUpDispatchVal.append(np.sum(np.sum(arrRegUpDispatch,axis=0)*lsNetru)/15)
    lsRegDownVal.append(np.sum(np.sum(arrRegDown,axis=0)*lsDAMrdMax))
    lsRegDownDispatchVal.append(np.sum(np.sum(arrRegDownDispatch,axis=0)*lsNetrd)/15)
               
    #produce graphs for each run
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegDownDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegDownDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegDownDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegDownDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Down Dispatch")
    plt.savefig("Output/Dispatch/regDownDispatch"+str(j)+".png")
    plt.close()
    
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegUpDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegUpDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegUpDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegUpDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Up Dispatch")
    plt.savefig("Output/Dispatch/regUpDispatch"+str(j)+".png")
    plt.close()
    
dfSummary=pd.DataFrame(list(zip(lsObj,lsRegUpVal,lsRegUpDispatchVal,lsRegDownVal,lsRegDownDispatchVal)),columns=["Obj Value","Regup Value","RegUpDispatch Value","Regdown Value","RegdownDispatch Value"])

#csv export of output values               
dfSummary.to_csv("Output/Dispatch/SummaryValues.csv",header=True)    
dfNumberofCycles.to_csv("Output/Dispatch/Cycles.csv",header=True)    
dfDegradation.to_csv("Output/Dispatch/Degradation.csv",header=True)  

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


#### Optimize for TOU only

In [13]:
constraints, varCharge, varSOCs = simulations.make_tou_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,\
                     fltDt, fltBatteryCap, arrConsumptionFinal, PeakLoads,intTotalNodes, dfNetNodeLoad,lsNodes,dctNodeIdentity)

tou_constraints, tou_obj_value,varCharge,varNumberOfCycles,varDegradationCost = simulations.make_tou_objectives(constraints, varCharge,arrSOCconstraint,fltDt,fltBatteryCap,lsCostElectric,arrConsumptionFinal)

touObj = cvx.Minimize(tou_obj_value)
touProb = cvx.Problem(touObj, tou_constraints)

touProb.solve(solver=cvx.MOSEK)

103.71347866626566

In [14]:
#graphs#Calculate flex values
lsFlex = []
arrCharge=np.array(varCharge.value)
for ind, strId in enumerate(dfNodesTrips['TripId']):
    
    lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()
    for day in range(int_run_days-1):
        lsLocations = lsLocations + lsLocations
 
    intStart = 0
    while intStart < 96*int_run_days and 'A' in lsLocations[intStart:]:

        # Find the first first time we start moving
        intStart = intStart + lsLocations[intStart:].index('A')

        # find how many time steps back we were charging
        intFlex = 0
        strLoc = 'H' # place holder
        while np.round(arrCharge[ind,intStart-intFlex],2) == 0 and strLoc != 'A':
            intFlex += 1
            strLoc = lsLocations[intStart-intFlex]
        lsFlex.append(intFlex)

        # from there find when we arrive at a destination
        if 'W' in lsLocations[intStart:] and 'H' in lsLocations[intStart:]:
            intEnd = min(lsLocations[intStart:].index('H'), lsLocations[intStart:].index('W'))
        elif 'W' in lsLocations[intStart:]:
            intEnd = lsLocations[intStart:].index('W')
        elif 'H' in lsLocations[intStart:]:
            intEnd = lsLocations[intStart:].index('H')
        else: # we are going to assume they are driving the rest of the time??
            break

        intStart += intEnd


In [15]:
#OUTPUTS
# THINGS TO SAVE !!

#objective val
print(tou_obj_value.value)
#charge over time (SOC and charge rate?)
arrCharging=np.array(varCharge.value)
arrSOC=np.array(varSOCs.value)

#Degradation Cost & Cycles
arrCycles=np.array(varNumberOfCycles.value)
arrDegradation=np.array(varDegradationCost.value) 
#time not charging prior to being away

arrFlex=np.array(lsFlex)


#output array files
np.savetxt(('Output/TOU/arrCharging.csv'),arrCharging, delimiter=",")
np.savetxt(('Output/TOU/arrSOC.csv'),arrSOC, delimiter=",")
np.savetxt(('Output/TOU/Cycles.csv'),arrCycles, delimiter=",")
np.savetxt(('Output/TOU/Degradation.csv'),arrDegradation, delimiter=",")
np.savetxt(('Output/TOU/FlexVals.csv'),arrFlex, delimiter=",")


103.71347866626564


#### Optimize a Stationary Battery

In [16]:
lsResNodes = [strNode for strNode in lsNodes if "Res" in strNode]
constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost = simulations.make_battery_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctResIdentity,lsResNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,lsDAMrdMax,lsDAMruMax,\
                     lsRuIdentity, lsRdIdentity,fltWorkRate)

bat_obj_value, varRegUp, varRegDown,varCharge = simulations.make_battery_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd,fltDt)


batObj = cvx.Maximize(bat_obj_value)
batProb = cvx.Problem(batObj, constraints)

batProb.solve(solver=cvx.MOSEK)

3115.2723316378615

In [17]:
#loop for dispatch quantity

lsObj=[]
lsCycles=[]
lsDegradation=[]
lsRegUpVal=[]
lsRegUpDispatchVal=[]
lsRegDownVal=[]
lsRegDownDispatchVal=[]

dfNumberofCycles=pd.DataFrame()
dfDegradation=pd.DataFrame()


for j in np.linspace(0,1,11):
    j=round(j,1)
    print(j)
    lsRuIdentity,lsRdIdentity = parameters.dispatches(int_run_days, int_run_hours, int_run_time_interval,j)
    lsResNodes = [strNode for strNode in lsNodes if "Res" in strNode]
    constraints, varRegDown,varRegUp,varCharge,varNumberOfCycles, varDegradationCost = simulations.make_battery_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge,dctResIdentity,lsResNodes, \
                     arrConsumptionFinal,PeakLoads,intTotalNodes,dfNetNodeLoad,\
                     fltBatteryCap,fltDt,lsCostElectric,lsDAMrdMax,lsDAMruMax,\
                     lsRuIdentity, lsRdIdentity,fltWorkRate)

    bat_obj_value, varRegUp, varRegDown,varCharge = simulations.make_battery_objectives(constraints, varRegDown,varRegUp,varCharge,varDegradationCost,\
                    lsCostElectric,lsRdIdentity, lsRuIdentity,lsDAMrdMax,lsDAMruMax,lsNetru,lsNetrd,fltDt)


    batObj = cvx.Maximize(bat_obj_value)
    batProb = cvx.Problem(batObj, constraints)

    batProb.solve(solver=cvx.MOSEK)
    
    ##File outputs
    
    #full arrays, export as csv directly
    arrRegUp=np.array(varRegUp.value)
    arrRegDown=np.array(varRegDown.value)
    arrCharging=np.array(varCharge.value)
    arrRegDownDispatch=varRegDown.value*lsRdIdentity
    arrRegUpDispatch=varRegUp.value*lsRuIdentity
    
        #output array files
    np.savetxt(('Output/Battery/Charging'+str(j)+'.csv'),arrCharging, delimiter=",")
    np.savetxt(('Output/Battery/arrRegUp'+str(j)+'.csv'),arrRegUp, delimiter=",")
    np.savetxt(('Output/Battery/arrRegDown'+str(j)+'.csv'),arrRegDown, delimiter=",")
    np.savetxt(('Output/Battery/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    np.savetxt(('Output/Battery/arrRegDownDispatch'+str(j)+'.csv'),arrRegDownDispatch, delimiter=",")
    
    #lists by each vehicle, add as an insert to datatframe
    lsCycles=varNumberOfCycles.value
    lsDegradation=varDegradationCost.value 
    
    #dataframe of list values
    dfNumberofCycles.insert(int(j*10),"Cycles"+str(j),lsCycles,True)
    dfDegradation.insert(int(j*10),"Degradation"+str(j),lsDegradation,True)
   
    #Individual values to save as list, these will zip into dataframe outside of loop
    lsObj.append(bat_obj_value.value)
    lsRegUpVal.append(np.sum(np.sum(arrRegUp,axis=0)*lsDAMruMax))
    lsRegUpDispatchVal.append(np.sum(np.sum(arrRegUpDispatch,axis=0)*lsNetru)/15)
    lsRegDownVal.append(np.sum(np.sum(arrRegDown,axis=0)*lsDAMrdMax))
    lsRegDownDispatchVal.append(np.sum(np.sum(arrRegDownDispatch,axis=0)*lsNetrd)/15)
               
    #produce graphs for each run
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegDownDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegDownDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegDownDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegDownDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Down Dispatch")
    plt.savefig("Output/Battery/regDownDispatch"+str(j)+".png")
    plt.close()
    
    fig, ax = plt.subplots(figsize=(11,7))
    for ind in range(1,np.shape(arrRegUpDispatch)[1]):

        if ind == 1: 
        
            ax.fill_between(np.arange(96*2),np.zeros(96*2),np.array(arrRegUpDispatch[ind,:]).flatten())
    
        else: 
            
            lb = np.sum(arrRegUpDispatch[0:ind,:],axis=0)
            ub = np.sum(arrRegUpDispatch[0:ind+1,:],axis=0)
       
            ax.fill_between(np.arange(96*2),np.array(lb).flatten(),np.array(ub).flatten())


    plt.ylabel('kW')
    plt.xlabel("Hour")
    plt.title("Regulation Up Dispatch")
    plt.savefig("Output/Battery/regUpDispatch"+str(j)+".png")
    plt.close()
    
dfSummary=pd.DataFrame(list(zip(lsObj,lsRegUpVal,lsRegUpDispatchVal,lsRegDownVal,lsRegDownDispatchVal)),columns=["Obj Value","Regup Value","RegUpDispatch Value","Regdown Value","RegdownDispatch Value"])

#csv export of output values               
dfSummary.to_csv("Output/Battery/SummaryValues.csv",header=True)    
dfNumberofCycles.to_csv("Output/Battery/Cycles.csv",header=True)    
dfDegradation.to_csv("Output/Battery/Degradation.csv",header=True)  

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
